## Import and set args

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from data.data import prepare_data
from collections import namedtuple
from utils import set_seed, Logger, CSVBatchLogger, log_args, add_args
import argparse
import os
import pandas as pd
import torch
from tqdm.auto import tqdm
from pprint import pprint

sampling_methods = ['subsample', 'reweight']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

parser = argparse.ArgumentParser()
parser.add_argument('--resnet_width', type=int, default=None)
parser.add_argument('--model_path', type=str)
parser.add_argument('--lr', type=float, default='1e-3')
parser.add_argument('--weight_decay', type=float, default='1e-6')
parser.add_argument('--sampling_method', choices=sampling_methods, default=None)
parser.add_argument('--n_epochs', type=int, default=25)
parser.add_argument('--log_dir', type=str, default='./phase2_log')
parser.add_argument('--seed', type=int, default=0)



# '--dataset', 'CelebA', '-s', 'confounder', '-t', 'Blond_Hair', '-c', 'Male', '--log_dir', 'twophaselog_w32', '--seed', '0', 
toparse = ['--resnet_width', '16',
          '--model_path', 'log_w16_seed0/last_model_50.pth',
          '--sampling_method', 'reweight']
args = parser.parse_args(toparse)

# setup logging
mode = 'w'
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)
logger = Logger(os.path.join(args.log_dir, 'log.txt'), mode)
# Record args
log_args(args, logger)
set_seed(args.seed)

Using cuda device
Resnet width: 16
Model path: log_w16_seed0/last_model_50.pth
Lr: 0.001
Weight decay: 1e-06
Sampling method: reweight
N epochs: 25
Log dir: ./phase2_log
Seed: 0



## Loads data
This is where we adjust the sampling strategy. 

In [2]:
from data.celebA_dataset import CelebADataset
from models import model_attributes
from data.dro_dataset import DRODataset

    
root_dir = '/home/thiennguyen/research/datasets/celebA/'  # dir that contains data
target_name= 'Blond_Hair'  # we are classifying whether the input image is blond or not
confounder_names= ['Male']  # we aim to avoid learning spurious features... here it's the gender
model_type= 'resnet10vw'  # what model we are using to process --> this is to determine the input size to rescale the image
augment_data= False
fraction=1.0
splits = ['train', 'val', 'test']
n_classes = 4

full_dataset = CelebADataset(root_dir=root_dir,
        target_name=target_name,
        confounder_names=confounder_names,
        model_type=model_type,  # this string is to get the model's input size (for resizing) and input type (image or precomputed)
        augment_data=augment_data)  # augment data adds random resized crop and random flip.

subsets = full_dataset.get_splits(       # basically return the Subsets object with the appropriate indices for train/val/test
        splits,                          # also implements subsampling --> just remove random indices of the appropriate groups in train
        train_frac=fraction,   # fraction means how much of the train data to use --> randomly remove if less than 1
        subsample_to_minority=(args.sampling_method == 'subsample'))

dro_subsets = [  
    DRODataset(
        subsets[split],  # process each subset separately --> applying the transform parameter.
        process_item_fn=None,
        n_groups=full_dataset.n_groups,
        n_classes=full_dataset.n_classes,
        group_str_fn=full_dataset.group_str) \
    for split in splits]

train_data, val_data, test_data = dro_subsets
train_loader = train_data.get_loader(train=True, reweight_groups=(args.sampling_method == 'reweight'), batch_size=128)
val_loader = val_data.get_loader(train=False, reweight_groups=None, batch_size=5)
test_loader = test_data.get_loader(train=False, reweight_groups=None, batch_size=128)

## Loads Models 

In [5]:
from variable_width_resnet import resnet10vw

# load saved model
model = resnet10vw(args.resnet_width, num_classes=n_classes)
# model.load_state_dict(torch.load(args.model_path))
model.to(device)

# freeze everything except the last layer
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

# make sure that really worked
parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias


# initialize the criterion and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=args.lr,
            momentum=0.9,
            weight_decay=args.weight_decay)

In [21]:
from variable_width_resnet import resnet10vw

model = resnet10vw(16, num_classes=4)
model2 = resnet10vw(16, num_classes=2)

model.to(device)
model2.to(device)

criteriontest = torch.nn.CrossEntropyLoss(reduction='none')

In [22]:


def pnorm(weights, p):
    normB = torch.norm(weights, 2, 1)
    ws = weights.clone()
    for i in range(weights.size(0)):
        ws[i] = ws[i] / torch.pow(normB[i], p)
    return ws


for batch_idx, batch in enumerate((val_loader)):
    batch = tuple(t.to(device) for t in batch)
    x,y,g = batch
    outputs4 = model(x)
    outputs2 = model2(x)
    loss4 = criteriontest(outputs4, g)
    loss2 = criteriontest(outputs2, y)
    break
    

In [23]:
group_idx = g 
n_groups = 4
losses = loss2

group_map = (group_idx == torch.arange(n_groups).unsqueeze(1).long().cuda()).float()
group_count = group_map.sum(1)
group_denom = group_count + (group_count==0).float() # avoid nans

group_loss = (group_map @ losses.view(-1))/group_denom
group_loss4 = (group_map @ loss4.view(-1))/group_denom

In [35]:
step_size = 0.01

adv_probs2 = torch.ones(n_groups).cuda()/n_groups
adv_probs4 = torch.ones(n_groups).cuda()/n_groups

adv_probs2 = adv_probs2 * torch.exp(step_size*group_loss)
print(adv_probs2.sum())
adv_probs2 = adv_probs2/adv_probs2.sum()

adv_probs4 = adv_probs4 * torch.exp(step_size*group_loss4)
print(adv_probs4.sum())
adv_probs4 = adv_probs4/adv_probs4.sum()

print(adv_probs2)

print(adv_probs4)

tensor(1.0032, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1.0069, device='cuda:0', grad_fn=<SumBackward0>)
tensor([0.2507, 0.2509, 0.2492, 0.2492], device='cuda:0',
       grad_fn=<DivBackward0>)
tensor([0.2518, 0.2517, 0.2483, 0.2483], device='cuda:0',
       grad_fn=<DivBackward0>)


In [25]:
print(f'x: {x.shape}\ny: {y}\ng: {g}')
print('group_map\n',group_map)
print(f'group_count\n{group_count}')
print(f'group_denom\n{group_denom}')
print(f'loss2\n{loss2}')
print(f'group_loss2\n{group_loss}')

print(f'loss4\n{loss4}')
print(f'group_loss4\n{group_loss4}')




x: torch.Size([5, 3, 224, 224])
y: tensor([0, 0, 0, 0, 0], device='cuda:0')
g: tensor([0, 1, 0, 1, 0], device='cuda:0')
group_map
 tensor([[1., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')
group_count
tensor([3., 2., 0., 0.], device='cuda:0')
group_denom
tensor([3., 2., 1., 1.], device='cuda:0')
loss2
tensor([0.6003, 0.6972, 0.6142, 0.6508, 0.6186], device='cuda:0',
       grad_fn=<NllLossBackward>)
group_loss2
tensor([0.6110, 0.6740, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<DivBackward0>)
loss4
tensor([1.3283, 1.2866, 1.3695, 1.4120, 1.5109], device='cuda:0',
       grad_fn=<NllLossBackward>)
group_loss4
tensor([1.4029, 1.3493, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<DivBackward0>)


## Train classifier

In [4]:
import csv
from pprint import pprint


def write_to_writer(writer, content):
    writer.writerow(content)
    pprint(content)

mode = 'w'
    
train_path = open(os.path.join(args.log_dir, 'train.csv'), mode)
val_path = open(os.path.join(args.log_dir, 'val.csv'), mode)
test_path = open(os.path.join(args.log_dir, 'test.csv'), mode)

train_columns = ['epoch','total_acc', 'group0_acc', 'group1_acc', 'group2_acc', 'group3_acc', 'split_acc', 'loss', 'avg_margin', 'group0_margin', 
                'group1_margin', 'group2_margin', 'group3_margin']

valtest_columns = ['epoch', 'total_acc', 'group0_acc', 'group1_acc', 'group2_acc', 'group3_acc', 'split_acc',
                  'avg_margin', 'group0_margin', 'group1_margin', 'group2_margin', 'group3_margin']

train_writer = csv.DictWriter(train_path, fieldnames=train_columns)
train_writer.writeheader()
val_writer = csv.DictWriter(val_path, fieldnames=valtest_columns)
val_writer.writeheader()
test_writer = csv.DictWriter(test_path, fieldnames=valtest_columns)
test_writer.writeheader()


def run_epoch(epoch, model, device, optimizer, loader, loss_computer, writer, is_training):
    """
    Train the classifier
    """
    model.train() if is_training else model.eval()
    
    running_loss, total_margin = 0, 0   # keep track of avg loss, margin in train
    l_correct, g_correct, total = 0, 0, 0  # for validation
    group_track = {f'g{i}':{'correct':0, 'total':0, 'margin': 0} for i in range(4)}  # keeps track of counts of #correct, #total for each group g0-g4
    log_train_every = 200    
    
    with (torch.enable_grad() if is_training else torch.no_grad()):
        for batch_idx, batch in enumerate(tqdm(loader)):
            batch = tuple(t.to(device) for t in batch)
            x,y,g = batch
            outputs = model(x)
            
            if is_training:
                optimizer.zero_grad()
                loss = loss_computer(outputs, g)  # we are classifying groups
                loss.backward()
                optimizer.step()
                # print statistics
                running_loss += loss.item()
                if (batch_idx % log_train_every) == log_train_every -1:    # print every 200 mini-batches
                    print('[%d, %5d] loss: %.3f, avg_margin: %.3f' %
                          (epoch, batch_idx + 1, running_loss / log_train_every, total_margin/total))
                    running_loss = 0.0
            
            # extra validation
            with torch.no_grad():
                # accuracies and margins
                total += g.size(0)  # total
                _, predicted = torch.max(outputs.data, 1)  # get predictions
                maskg = torch.zeros((len(g), 4), dtype=bool, device=device)
                maskg[np.arange(len(g)),g] = 1
                margins = outputs[maskg]-torch.max(outputs*(~maskg),dim=1)[0]
                total_margin += margins.sum().item()
                for g_idx in range(4):
                    group_track[f'g{g_idx}']['correct'] += ((predicted == g)[g==g_idx]).sum().item()  # correctly predict
                    group_track[f'g{g_idx}']['total'] += (g==g_idx).sum().item()  # total number of group's instance encountered
                    group_track[f'g{g_idx}']['margin'] += margins[g==g_idx].sum().item()
                g_correct += (predicted == g).sum().item()
                l_correct += (predicted//2 == y).sum().item()

        # write stats in dict and csv
        stats_dict = {'epoch':epoch, 'total_acc':f"{l_correct/total:.4f}", 'split_acc':f'{g_correct/total:.4f}', 'avg_margin': f"{total_margin/total:.4f}"}
        for g in range(4):
            stats_dict[f'group{g}_acc'] = f"{group_track[f'g{g}']['correct']/group_track[f'g{g}']['total']:.4f}"
            stats_dict[f'group{g}_margin'] = f"{group_track[f'g{g}']['margin']/group_track[f'g{g}']['total']:.4f}"
        if is_training:
            stats_dict['loss'] = f'{running_loss/log_train_every:.4f}'
            
        write_to_writer(writer, stats_dict)

In [5]:
for epoch in range(args.n_epochs):
    # train
    print(f'Train epoch {epoch}')
    run_epoch(epoch+1, model, device, optimizer, train_loader, criterion, train_writer, is_training=True)
    
    # validate
    print(f'Validate epoch {epoch}')
    run_epoch(epoch+1, model, device, optimizer, val_loader, criterion, val_writer, is_training=False)
    

Train epoch 0


[1,   200] loss: 0.280, avg_margin: 3.858
[1,   400] loss: 0.188, avg_margin: 3.879
[1,   600] loss: 0.189, avg_margin: 3.886
[1,   800] loss: 0.184, avg_margin: 3.898
[1,  1000] loss: 0.181, avg_margin: 3.908
[1,  1200] loss: 0.187, avg_margin: 3.914

{'avg_margin': '3.9181',
 'epoch': 1,
 'group0_acc': '0.9403',
 'group0_margin': '5.0920',
 'group1_acc': '0.9267',
 'group1_margin': '3.2736',
 'group2_acc': '0.9397',
 'group2_margin': '4.6347',
 'group3_acc': '0.8971',
 'group3_margin': '2.6794',
 'loss': '0.0651',
 'split_acc': '0.9259',
 'total_acc': '0.9342'}
Validate epoch 0



{'avg_margin': '3.9809',
 'epoch': 1,
 'group0_acc': '0.8901',
 'group0_margin': '4.7389',
 'group1_acc': '0.9033',
 'group1_margin': '3.1593',
 'group2_acc': '0.8744',
 'group2_margin': '4.2288',
 'group3_acc': '0.7637',
 'group3_margin': '1.8769',
 'split_acc': '0.8922',
 'total_acc': '0.9161'}
Train epoch 1


[2,   200] loss: 0.183, avg_margin: 3.970
[2,   400] loss: 0.184, avg_margin: 3.985
[2,   600] loss: 0.184, avg_margin: 3.985
[2,   800] loss: 0.185, avg_margin: 3.987
[2,  1000] loss: 0.184, avg_margin: 3.994
[2,  1200] loss: 0.182, avg_margin: 4.002

{'avg_margin': '4.0036',
 'epoch': 2,
 'group0_acc': '0.9373',
 'group0_margin': '5.0494',
 'group1_acc': '0.9268',
 'group1_margin': '3.3377',
 'group2_acc': '0.9487',
 'group2_margin': '4.7137',
 'group3_acc': '0.9179',
 'group3_margin': '2.9146',
 'loss': '0.0670',
 'split_acc': '0.9327',
 'total_acc': '0.9406'}
Validate epoch 1



{'avg_margin': '4.1527',
 'epoch': 2,
 'group0_acc': '0.8887',
 'group0_margin': '4.8049',
 'group1_acc': '0.9083',
 'group1_margin': '3.4658',
 'group2_acc': '0.8824',
 'group2_margin': '4.3447',
 'group3_acc': '0.7582',
 'group3_margin': '1.7690',
 'split_acc': '0.8948',
 'total_acc': '0.9191'}
Train epoch 2


...


{'avg_margin': '4.4776',
 'epoch': 23,
 'group0_acc': '0.8838',
 'group0_margin': '5.1639',
 'group1_acc': '0.9097',
 'group1_margin': '3.7621',
 'group2_acc': '0.8817',
 'group2_margin': '4.6493',
 'group3_acc': '0.7912',
 'group3_margin': '2.1138',
 'split_acc': '0.8934',
 'total_acc': '0.9173'}
Train epoch 23


[24,   200] loss: 0.170, avg_margin: 4.503
[24,   400] loss: 0.177, avg_margin: 4.484
[24,   600] loss: 0.176, avg_margin: 4.489
[24,   800] loss: 0.171, avg_margin: 4.494
[24,  1000] loss: 0.170, avg_margin: 4.497
[24,  1200] loss: 0.174, avg_margin: 4.497

{'avg_margin': '4.4964',
 'epoch': 24,
 'group0_acc': '0.9389',
 'group0_margin': '5.7422',
 'group1_acc': '0.9297',
 'group1_margin': '3.8181',
 'group2_acc': '0.9529',
 'group2_margin': '5.1346',
 'group3_acc': '0.9229',
 'group3_margin': '3.2812',
 'loss': '0.0634',
 'split_acc': '0.9361',
 'total_acc': '0.9440'}
Validate epoch 23



{'avg_margin': '4.5605',
 'epoch': 24,
 'group0_acc': '0.8890',
 'group0_margin': '5.3041',
 'group1_acc': '0.9122',
 'group1_margin': '3.8280',
 'group2_acc': '0.8747',
 'group2_margin': '4.6214',
 'group3_acc': '0.7692',
 'group3_margin': '2.0421',
 'split_acc': '0.8955',
 'total_acc': '0.9196'}
Train epoch 24


[25,   200] loss: 0.166, avg_margin: 4.544
[25,   400] loss: 0.174, avg_margin: 4.536
[25,   600] loss: 0.178, avg_margin: 4.521
[25,   800] loss: 0.177, avg_margin: 4.515
[25,  1000] loss: 0.172, avg_margin: 4.516
[25,  1200] loss: 0.173, avg_margin: 4.519

{'avg_margin': '4.5196',
 'epoch': 25,
 'group0_acc': '0.9405',
 'group0_margin': '5.7445',
 'group1_acc': '0.9271',
 'group1_margin': '3.8385',
 'group2_acc': '0.9536',
 'group2_margin': '5.1837',
 'group3_acc': '0.9242',
 'group3_margin': '3.3095',
 'loss': '0.0610',
 'split_acc': '0.9364',
 'total_acc': '0.9445'}
Validate epoch 24



{'avg_margin': '4.6103',
 'epoch': 25,
 'group0_acc': '0.8909',
 'group0_margin': '5.3639',
 'group1_acc': '0.9142',
 'group1_margin': '3.9133',
 'group2_acc': '0.8716',
 'group2_margin': '4.5473',
 'group3_acc': '0.7637',
 'group3_margin': '1.9607',
 'split_acc': '0.8967',
 'total_acc': '0.9208'}
